<a href="https://colab.research.google.com/github/jeremymoynihan/EA-Soccer/blob/main/FC24_Player_Data_Acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
##Code to Export all Player Data

import requests
import pandas as pd
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def fetch_data(base_url, initial_offset=0, limit=100):
    """
    Fetch data from the given base URL, paginating through the results.

    Parameters:
        base_url (str): The base URL for the API request with a placeholder for offset.
        initial_offset (int): The starting point for data fetching.
        limit (int): The number of items to fetch per request.

    Returns:
        pd.DataFrame: A DataFrame containing all fetched data.
    """
    all_players_df = pd.DataFrame()
    offset = initial_offset

    while True:
        url = base_url.format(offset=offset)
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if not data.get('items'):
                logging.info("No more data to fetch.")
                break

            df = pd.json_normalize(data['items'])
            all_players_df = pd.concat([all_players_df, df], ignore_index=True)
            offset += limit

        except requests.exceptions.HTTPError as http_err:
            logging.error(f"HTTP error occurred: {http_err}")
            break
        except requests.exceptions.ConnectionError as conn_err:
            logging.error(f"Connection error occurred: {conn_err}")
            break
        except requests.exceptions.Timeout as timeout_err:
            logging.error(f"Timeout error occurred: {timeout_err}")
            break
        except requests.exceptions.RequestException as req_err:
            logging.error(f"Request error occurred: {req_err}")
            break
        except ValueError as val_err:
            logging.error(f"JSON decoding failed: {val_err}")
            break

    return all_players_df

def main():
    """
    Main function to fetch data from the API and save it to a CSV file.
    """
    base_url = "https://drop-api.ea.com/rating/fc-24?locale=en&offset={offset}&limit=100"
    all_players_df = fetch_data(base_url)

    if not all_players_df.empty:
        logging.info("Fetched player data:")
        print(all_players_df)

        csv_file_path = 'fc-24.csv'
        all_players_df.to_csv(csv_file_path, index=False)
        logging.info(f"Data saved to {csv_file_path}")
    else:
        logging.info("No data was fetched to save.")

if __name__ == "__main__":
    main()

           id   rank  overallRating firstName         lastName  \
0      231747      1             91    Kylian           Mbappé   
1      227203      2             91    Alexia  Putellas Segura   
2      239085      3             91    Erling          Haaland   
3      192985      4             91     Kevin        De Bruyne   
4      241667      5             90    Aitana    Bonmatí Conca   
...       ...    ...            ...       ...              ...   
17321  266248  17318             47     Deven          Sawhney   
17322  258959  17318             47    Tengda             Wang   
17323  269541  17318             47    Yuhang               Wu   
17324  275671  17318             47    Daoxin               Ye   
17325  269534  17318             47       Jie              Yin   

            commonName               birthdate  height  skillMoves  \
0                 None  12/20/1998 12:00:00 AM     182           5   
1      Alexia Putellas    2/4/1994 12:00:00 AM     173           5 